In [1]:
import csv
import datetime
from operator import sub
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing, ensemble
import sys


In [40]:
feature_cols = ["employee_index","country_residence","sex","age","first_join_date","new_customer_index","seniority","primary","customer_type","customer_relation","residence_idx","foreigner_idx","channel","deceased","address","province_code","province_name","activity_idx","gross_income","segment"]
#target_cols=["saving_account","guarantees","current_account","derivada_account","payroll_account","junior_account","mas_particular_account","particular_account","particular_plus_account","short-term_deposits","medium-term_deposits","long-term_deposits","e-account","funds","mortgage","pensions","loans","taxes","credit_card","securities","home_account","payroll","pensions_1","direct_debit"]
#target_cols = target_cols[2:]
target_cols = ["current_account"]
all_cols = feature_cols+target_cols


In [39]:
len(all_cols)

21

In [41]:
def getTarget(row):
    tlist = []
    for col in target_cols:
            if row[col].strip() in ['', 'NA']:
                    target = 0
            else:
                    target = int(float(row[col]))
            tlist.append(target)
    return tlist


In [106]:
def processData(in_file_name):
    x_vars_list = []
    #y_vars_list = []
    for row in csv.DictReader(in_file_name, fieldnames = ( "date","id","employee_index","country_residence","sex",
                                                          "age","first_join_date","new_customer_index","seniority",
                                                          "primary","last_date_primary","customer_type","customer_relation",
                                                          "residence_idx","foreigner_idx","spouse_idx","channel","deceased",
                                                          "address","province_code","province_name","activity_idx",
                                                          "gross_income","segment","saving_account","guarantees",
                                                          "current_account","derivada_account","payroll_account",
                                                          "junior_account","mas_particular_account","particular_account",
                                                          "particular_plus_account","short-term_deposits","medium-term_deposits",
                                                          "long-term_deposits","e-account","funds","mortgage","pensions","loans",
                                                          "taxes","credit_card","securities","home_account","payroll",
                                                          "pensions_1","direct_debit" )):
        x_vars = []
        for col in feature_cols:
            val = row[col].strip()
            x_vars.append(val)
            
        if row['date'] in [ '2016-05-28']:
            target_list = getTarget(row)
            x_vars_list.append(x_vars + target_list)
            
#assign a number for the test data
        elif row['date'] in ['2016-06-28']: 
             target_list = [0]
             x_vars_list.append(x_vars + target_list)
        else:
            continue

    return x_vars_list

In [44]:
len(x_vars_list[1000])

20

In [97]:
def runXGB(train_X, train_y, seed_val=0):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.05
    param['max_depth'] = 8
    param['silent'] = 1
    param['num_class'] = 2
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = 50

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    model = xgb.train(plst, xgtrain, num_rounds)
    return model


In [46]:
len(all_cols)

21

In [38]:
def print_uniq(df):
    feature_cols = ["employee_index","country_residence","sex","age","first_join_date","new_customer_index","seniority",
    "primary","customer_type","customer_relation","residence_idx","foreigner_idx","channel","deceased","address",
    "province_code","province_name","activity_idx","gross_income","segment"]
    feature_cols_subset = feature_cols
    feature_cols_subset.remove("gross_income")
    for ind, feature_col in enumerate(feature_cols_subset):

        uniq_values = list(df[feature_col].unique())

        print("uniq values for "+feature_col+": ",uniq_values)
        print("number of uniq values for "+feature_col+": ",len(uniq_values))

In [107]:
if __name__ == "__main__":
    start_time = datetime.datetime.now()
    data_path = "./input/"
    #train_file =  open(data_path + "train_ver2.csv")
    train_file =  open(data_path + "train_20percent.csv")
    x_vars_list = processData(train_file)
    

    df_org = pd.DataFrame.from_records(x_vars_list, columns=all_cols)
    df = df_org
    print("Number of columns in the dataframe:")
    print(len(x_vars_list[1]))  
    print("Number of records in the dataframe:")
    print(len(df))
    

Number of columns in the dataframe:
21
Number of records in the dataframe:
185957


In [108]:
print("Print uniq values for each column in the dataframe:")
print_uniq(df)


Print uniq values for each column in the dataframe:
('uniq values for employee_index: ', ['N', 'A', 'S', 'F', 'B'])
('number of uniq values for employee_index: ', 5)
('uniq values for country_residence: ', ['ES', 'BE', 'IN', 'AR', 'AE', 'FR', 'VE', 'BO', 'UY', 'US', 'DO', 'BR', 'DE', 'PY', 'GT', 'LT', 'CH', 'CL', 'RO', 'GB', 'CO', 'IL', 'NL', 'MX', 'IT', 'AT', 'GQ', 'PT', 'SA', 'BG', 'FI', 'GN', 'AU', 'CU', 'MD', 'EC', 'MA', 'UA', 'PA', 'NO', 'PE', 'SK', 'BA', 'CF', 'LU', 'GR', 'CR', 'HN', 'AL', 'GW', 'SE', 'PL', 'IE', 'CA', 'NI', 'PH', 'EG', 'VN', 'RU', 'MM', 'GA', 'JP', 'BY', 'KE', 'NG', 'CZ', 'PK', 'CN', 'MK', 'LY', 'SG', 'TW', 'SN', 'SV', 'AD', 'LB', 'CM'])
('number of uniq values for country_residence: ', 77)
('uniq values for sex: ', ['H', 'V', ''])
('number of uniq values for sex: ', 3)
('uniq values for age: ', ['42', '31', '34', '44', '38', '36', '52', '39', '74', '61', '41', '29', '45', '32', '37', '70', '48', '77', '22', '33', '51', '65', '63', '30', '43', '64', '76', '35', 

In [109]:
def prep_numeric_data(df):
#Age and seniority may have non-numeric values. Use errors='coerce' to force them to be NaN so they can be removed.
    df['age']   = pd.to_numeric(df['age'], errors='coerce')
    df['seniority']   = pd.to_numeric(df['seniority'], errors='coerce')
    ##df = df[df['seniority']>0]
    ##df.dropna(axis=0)
    #df.dropna()
    #print(df.isnull().sum())
#select the rows that do not have NaN
    condition=~pd.isnull(df).any(axis=1)
    df=df[condition]
    
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    feature_category_cols = feature_cols
    for ind, feature_col in enumerate(feature_category_cols):
        le.fit(df[feature_col])
        df[feature_col]=le.transform(df[feature_col])
    
    return df

In [110]:
df = prep_numeric_data(df)


In [111]:
print("Print uniq values for each column after label encoding:")
print_uniq(df)

Print uniq values for each column after label encoding:
('uniq values for employee_index: ', [3, 0, 4, 2, 1])
('number of uniq values for employee_index: ', 5)
('uniq values for country_residence: ', [26, 7, 39, 3, 1, 28, 75, 9, 74, 73, 23, 10, 22, 62, 34, 44, 14, 15, 63, 30, 18, 38, 54, 51, 40, 4, 32, 61, 65, 8, 27, 31, 5, 20, 48, 24, 47, 72, 56, 55, 57, 68, 6, 13, 45, 33, 19, 36, 2, 35, 66, 60, 37, 12, 53, 58, 25, 76, 64, 50, 29, 41, 11, 42, 52, 21, 59, 17, 49, 46, 67, 71, 69, 70, 0, 43, 16])
('number of uniq values for country_residence: ', 77)
('uniq values for sex: ', [1, 2, 0])
('number of uniq values for sex: ', 3)
('uniq values for age: ', [40, 29, 32, 42, 36, 34, 50, 37, 72, 59, 39, 27, 43, 30, 35, 68, 46, 75, 20, 31, 49, 63, 61, 28, 41, 62, 74, 33, 55, 38, 60, 89, 44, 76, 45, 58, 65, 88, 51, 69, 79, 48, 54, 70, 78, 24, 52, 47, 83, 17, 71, 98, 64, 86, 9, 53, 56, 66, 95, 81, 15, 67, 90, 14, 57, 77, 82, 85, 84, 73, 25, 22, 80, 91, 92, 103, 21, 23, 87, 18, 94, 26, 16, 10, 11, 93,

In [112]:
print(df[0:2])

   employee_index  country_residence  sex  age  first_join_date  \
0               3                 26    1   40             3378   
1               3                 26    2   29             3378   

   new_customer_index  seniority  primary  customer_type  customer_relation  \
0                   0        115        0              1                  1   
1                   0        115        0              1                  2   

        ...         foreigner_idx  channel  deceased  address  province_code  \
0       ...                     0       33         0        0             21   
1       ...                     0      119         0        0             21   

   province_name  activity_idx  gross_income  segment  current_account  
0             31             1         22542        2                1  
1             31             0         38436        2                0  

[2 rows x 21 columns]


In [122]:
train = df.drop(['address','current_account'],axis=1)
target = df['current_account']

#test = pd.read_csv("test.csv")
#test = test.drop(['ID'],axis=1)

#xgtrain = xgb.DMatrix(train.values, target.values)
#xgtest = xgb.DMatrix(test.values)

train_X = train.values
train_y = target.values

##xgtrain = xgb.DMatrix(train_X, label=train_y)
print("Building xgboost model...")
print("Number of training data:")
print(train_X.shape)
model = runXGB(train_X, train_y, seed_val=0)
del train_X, train_y


test_file = open(data_path + "test_ver2.csv")
x_vars_list = processData(test_file)
df_test = pd.DataFrame.from_records(x_vars_list, columns=all_cols)

df_test = prep_numeric_data(df_test)
df_test = df_test.drop(['address','current_account'],axis=1)
test_X = df_test.values
print("Number of test data:")
print(test_X.shape)
print("Predicting...")
xgtest = xgb.DMatrix(test_X)
preds = model.predict(xgtest)

#del test_X, xgtest
test_file.close()
print("Time used for the model:")
print(datetime.datetime.now()-start_time)

Building xgboost model...
Number of training data:
(185957L, 19L)
Number of test data:
(929615L, 19L)
Predicting...
Time used for the model:
0:28:29.645000


In [123]:
print(preds.shape)

(929615L, 2L)


In [127]:
preds[0:20]


array([[ 0.34774917,  0.65225083],
       [ 0.71539527,  0.28460473],
       [ 0.12499154,  0.87500846],
       [ 0.27747777,  0.72252226],
       [ 0.24046989,  0.75953013],
       [ 0.26894912,  0.73105085],
       [ 0.30498889,  0.69501114],
       [ 0.26255068,  0.73744929],
       [ 0.43130919,  0.56869084],
       [ 0.12369116,  0.8763088 ],
       [ 0.19044708,  0.80955291],
       [ 0.42271209,  0.57728791],
       [ 0.27795956,  0.72204041],
       [ 0.19403829,  0.80596179],
       [ 0.22575028,  0.77424973],
       [ 0.18864547,  0.81135452],
       [ 0.19814488,  0.80185509],
       [ 0.74926633,  0.25073367],
       [ 0.34575841,  0.65424162],
       [ 0.42271209,  0.57728791]], dtype=float32)